In [1]:
from __future__ import division, print_function
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

%cd HepG2

/users/avanti/interpret-benchmark/data/HepG2


In [9]:
%matplotlib inline

import keras
from keras.models import model_from_json
from keras.models import Model
import h5py
import numpy as np
from deeplift import dinuc_shuffle
from matplotlib import pyplot as plt
import sys
from deeplift.visualization import viz_sequence
import scipy
from scipy import stats


argmax_to_letter = {0:'A', 1:'C', 2:'G', 3:'T'}
def onehot_to_seq(onehot):
    seq = "".join([argmax_to_letter[x] for x in np.argmax(onehot,axis=-1)])
    return seq

def one_hot_encode_along_channel_axis(sequence):
    to_return = np.zeros((len(sequence),4), dtype=np.int8)
    seq_to_one_hot_fill_in_array(zeros_array=to_return,
                                 sequence=sequence, one_hot_axis=1)
    return to_return

def seq_to_one_hot_fill_in_array(zeros_array, sequence, one_hot_axis):
    assert one_hot_axis==0 or one_hot_axis==1
    if (one_hot_axis==0):
        assert zeros_array.shape[1] == len(sequence)
    elif (one_hot_axis==1): 
        assert zeros_array.shape[0] == len(sequence)
    #will mutate zeros_array
    for (i,char) in enumerate(sequence):
        if (char=="A" or char=="a"):
            char_idx = 0
        elif (char=="C" or char=="c"):
            char_idx = 1
        elif (char=="G" or char=="g"):
            char_idx = 2
        elif (char=="T" or char=="t"):
            char_idx = 3
        elif (char=="N" or char=="n"):
            continue #leave that pos as all 0's
        else:
            raise RuntimeError("Unsupported character: "+str(char))
        if (one_hot_axis==0):
            zeros_array[char_idx,i] = 1
        elif (one_hot_axis==1):
            zeros_array[i,char_idx] = 1

def compare_methods(method_1_name, method_2_name, score_lookup):
    method_1_scores=score_lookup[method_1_name]
    method_2_scores=score_lookup[method_2_name]
    #plt.scatter(method_1_scores, method_2_scores, alpha=0.3)
    #plt.plot([np.min(method_1_scores), np.max(method_1_scores)],
    #         [np.min(method_1_scores), np.max(method_1_scores)],
    #         color="black")
    #plt.xlabel(method_1_name)
    #plt.ylabel(method_2_name)
    #plt.show()
    differences = method_2_scores-method_1_scores
    sorted_differences = sorted(differences,key=lambda x:np.abs(x))
    #print(sorted_differences)
    positive_ranks = sum([x[0]+1 for x in enumerate(sorted_differences) if x[1] > 0])
    negative_ranks = sum([x[0]+1 for x in enumerate(sorted_differences) if x[1] < 0])
    wilcoxon_result = scipy.stats.wilcoxon(differences)
    if (wilcoxon_result.pvalue > 0.05):
        symbol = "~"
    else:
        symbol = ">"
    
    if (negative_ranks < positive_ranks):
        print(method_2_name," "+symbol+" ",method_1_name,"pval",wilcoxon_result.pvalue)
    else:
        print(method_1_name," "+symbol+" ",method_2_name,"pval",wilcoxon_result.pvalue)
    

In [10]:

#dan preinit
#model_prefix = "record_21_model_NZqHC"
#model_prefix = "record_20_model_QFbAM"

model_prefixes_and_folders = [
    ("record_21_model_NZqHC","imp_scores/danpreinit"),
    ("record_20_model_QFbAM","imp_scores/danpreinit"),
    ("record_19_model_h9WFI","imp_scores/danpreinit"),
    ("record_18_model_u5eMJ","imp_scores/danpreinit"),
    ("record_17_model_Fljbs","imp_scores/danpreinit"),
    ("record_16_model_tSI7v","imp_scores/danpreinit"),
    ("record_14_model_g60lE","imp_scores/danpreinit"),
    ("record_11_model_0XKud","imp_scores/danpreinit"),
    ("record_7_model_2ObnU","imp_scores/danpreinit"),
    ("record_4_model_Dvrnb","imp_scores/danpreinit"),

    ("record_15_model_REDP7","imp_scores/fromscratch"),
    ("record_13_model_FXPOF","imp_scores/fromscratch"),
    ("record_12_model_rjGet","imp_scores/fromscratch"),
    ("record_10_model_audzA","imp_scores/fromscratch"),
    ("record_9_model_ROCRa","imp_scores/fromscratch"),
    ("record_8_model_qyto7","imp_scores/fromscratch"),
    ("record_6_model_svFYA","imp_scores/fromscratch"),
    ("record_5_model_1Q9Lp","imp_scores/fromscratch"),
    ("record_3_model_0FT6i","imp_scores/fromscratch"),
    ("record_2_model_IH83H","imp_scores/fromscratch"),               
]


#trained from scratch
#model_prefix = "record_15_model_REDP7"

#fcfromscratch
#model_prefix = "record_25_model_Fq78N"
#imp_scores_folder = "imp_scores/fcfromscratch-randinporder"

model_prefix_to_subbed_diffs_lookup = {}
model_prefix_to_zerod_diffs_lookup = {}

for model_prefix,imp_scores_folder in model_prefixes_and_folders:
    print("\n\n###########\nOn model prefix",model_prefix,"\n############")
    
    model_weights = "model_files/"+model_prefix+"_modelWeights.h5"
    model_json = "model_files/"+model_prefix+"_modelJson.json"

    keras_model = model_from_json(open(model_json).read())
    keras_model.load_weights(model_weights)

    preact_model = Model(inputs=keras_model.input,
                         outputs=keras_model.layers[-2].output)

    imp_scores_fh = h5py.File(imp_scores_folder+"/imp_scores_"+model_prefix+".h5", "r")
    onehot_seq = np.array(imp_scores_fh['onehot'])
    orig_preds = preact_model.predict(onehot_seq).squeeze()

    seqs = [onehot_to_seq(x) for x in onehot_seq]
    shuffled_onehot_seqs = []
    num_shuffles_to_make = 10
    for i in range(num_shuffles_to_make):
        shuffled_onehot_seqs.append(np.array(
                [one_hot_encode_along_channel_axis(dinuc_shuffle.dinuc_shuffle(x)) for x in seqs]))
    

    #######Score things
    method_to_zerod_diffs = {}
    method_to_subbed_diffs = {}

    percentile = 80

    scoring_methods = [
        'scores_ism',
        'scores_deeplift_rescale_dinucshuff',
        'scores_deeplift_genomicsdefault_dinucshuff',
        'scores_integrated_grad20_dinucshuff',
        'scores_grad_times_inp_flatref',

        'scores_integrated_grad20_avgposref',
        'scores_deeplift_rescale_avgposref',
        'scores_deeplift_genomicsdefault_avgposref',
        'scores_integrated_grad20_flatref',
        'scores_deeplift_rescale_flatref',
        'scores_deeplift_genomicsdefault_flatref',

        'scores_integrated_grad10_dinucshuff',
        'scores_integrated_grad2_dinucshuff',
        'scores_integrated_grad5_dinucshuff',     
    ]

    for method in scoring_methods:
        #print(method)
        sys.stdout.flush()
        scores = np.array(imp_scores_fh[method])
        central_scores = scores[:,350:650]

        #for each example, zero out the top 10% of scores

        #to_zero = list(zip(*np.nonzero(central_scores > np.percentile(central_scores,percentile))))
        #print(len(to_zero))

        to_zero = [
            (i,x) for i,ex_central_scores in enumerate(central_scores)
             for x in np.nonzero(ex_central_scores >
                                 np.percentile(ex_central_scores,percentile))[0]
        ]
        #print(len(to_zero))

        zerod_inputs = np.array(onehot_seq)
        for example_idx, col_idx in to_zero:
            zerod_inputs[example_idx,350+col_idx] = 0.25

        subbed_inputs_sets = []
        for i in range(num_shuffles_to_make):
            subbed_inputs = np.array(onehot_seq)
            for example_idx, col_idx in to_zero:
                subbed_inputs[example_idx,350+col_idx] = shuffled_onehot_seqs[i][example_idx,350+col_idx]
            subbed_inputs_sets.append(subbed_inputs)
        mean_subbed_preds = np.mean(np.array([preact_model.predict(x).squeeze() for x in subbed_inputs_sets]),axis=0)
        method_to_subbed_diffs[method] = orig_preds-mean_subbed_preds
        print("\navg_diff_meansubbed",method,np.mean(orig_preds-mean_subbed_preds),"num negative:",np.sum(orig_preds-mean_subbed_preds < 0))

        #viz_sequence.plot_weights((onehot_seq[0]*scores[0][:,None])[350:650])
        #viz_sequence.plot_weights((altered_inputs[0]*scores[0][:,None])[350:650])

        zerod_preds = np.squeeze(preact_model.predict(zerod_inputs))
        method_to_zerod_diffs[method] = orig_preds-zerod_preds
        print("avg_diff_zerod",method,np.mean(orig_preds-zerod_preds),"num negative:",np.sum(orig_preds-zerod_preds < 0))

        sys.stdout.flush()
    
    ########Run comparisons
    
    model_prefix_to_subbed_diffs_lookup[model_prefix] = method_to_subbed_diffs
    model_prefix_to_zerod_diffs_lookup[model_prefix] = method_to_zerod_diffs

    diffs_lookup = method_to_subbed_diffs    

    #print("Compare to ISM")
    print("\n##Comparisons##\n")

    compare_methods(method_1_name="scores_ism",
                    method_2_name="scores_deeplift_genomicsdefault_dinucshuff",
                    score_lookup=diffs_lookup)
    compare_methods(method_1_name="scores_ism",
                    method_2_name="scores_deeplift_rescale_dinucshuff",
                    score_lookup=diffs_lookup)
    compare_methods(method_1_name="scores_ism",
                    method_2_name="scores_integrated_grad20_dinucshuff",
                    score_lookup=diffs_lookup)
    compare_methods(method_1_name="scores_ism",
                    method_2_name="scores_grad_times_inp_flatref",
                    score_lookup=diffs_lookup)

    #print("Comparisons to DeepLIFT")

    compare_methods(method_1_name="scores_integrated_grad20_dinucshuff",
                    method_2_name="scores_deeplift_genomicsdefault_dinucshuff",
                    score_lookup=diffs_lookup)
    compare_methods(method_1_name="scores_grad_times_inp_flatref",
                    method_2_name="scores_deeplift_genomicsdefault_dinucshuff",
                    score_lookup=diffs_lookup)
    
    compare_methods(method_1_name="scores_integrated_grad20_dinucshuff",
                    method_2_name="scores_deeplift_rescale_dinucshuff",
                    score_lookup=diffs_lookup)
    compare_methods(method_1_name="scores_grad_times_inp_flatref",
                    method_2_name="scores_deeplift_rescale_dinucshuff",
                    score_lookup=diffs_lookup)

    #print("Deeplift revealcancel vs rescale")

    compare_methods(method_1_name="scores_deeplift_rescale_dinucshuff",
                    method_2_name="scores_deeplift_genomicsdefault_dinucshuff",
                    score_lookup=diffs_lookup)

    #print("Grad times inp vs intgrad")
    compare_methods(method_1_name="scores_grad_times_inp_flatref",
                    method_2_name="scores_integrated_grad20_dinucshuff",
                    score_lookup=diffs_lookup)
    
    #print("Compare references")
    compare_methods(method_1_name="scores_integrated_grad20_flatref",
                    method_2_name="scores_integrated_grad20_dinucshuff",
                    score_lookup=diffs_lookup)
    compare_methods(method_1_name="scores_integrated_grad20_avgposref",
                    method_2_name="scores_integrated_grad20_dinucshuff",
                    score_lookup=diffs_lookup)
    compare_methods(method_1_name="scores_deeplift_rescale_flatref",
                    method_2_name="scores_deeplift_rescale_dinucshuff",
                    score_lookup=diffs_lookup)
    compare_methods(method_1_name="scores_deeplift_rescale_avgposref",
                    method_2_name="scores_deeplift_rescale_dinucshuff",
                    score_lookup=diffs_lookup)
    compare_methods(method_1_name="scores_deeplift_genomicsdefault_flatref",
                    method_2_name="scores_deeplift_genomicsdefault_dinucshuff",
                    score_lookup=diffs_lookup)
    compare_methods(method_1_name="scores_deeplift_genomicsdefault_avgposref",
                    method_2_name="scores_deeplift_genomicsdefault_dinucshuff",
                    score_lookup=diffs_lookup)

    #print("Compare different intgrads)
    compare_methods(method_1_name="scores_integrated_grad2_dinucshuff",
                    method_2_name="scores_integrated_grad20_dinucshuff",
                    score_lookup=diffs_lookup)
    compare_methods(method_1_name="scores_integrated_grad10_dinucshuff",
                    method_2_name="scores_integrated_grad20_dinucshuff",
                    score_lookup=diffs_lookup)




###########
On model prefix record_21_model_NZqHC 
############

avg_diff_meansubbed scores_ism 32.737465 num negative: 7
avg_diff_zerod scores_ism 6.9883213 num negative: 407

avg_diff_meansubbed scores_deeplift_rescale_dinucshuff 31.819494 num negative: 4
avg_diff_zerod scores_deeplift_rescale_dinucshuff 5.1905246 num negative: 425

avg_diff_meansubbed scores_deeplift_genomicsdefault_dinucshuff 31.282904 num negative: 3
avg_diff_zerod scores_deeplift_genomicsdefault_dinucshuff 5.6305656 num negative: 412

avg_diff_meansubbed scores_integrated_grad20_dinucshuff 22.657883 num negative: 30
avg_diff_zerod scores_integrated_grad20_dinucshuff 9.405571 num negative: 341

avg_diff_meansubbed scores_grad_times_inp_flatref 26.736555 num negative: 24
avg_diff_zerod scores_grad_times_inp_flatref 2.1125126 num negative: 508

avg_diff_meansubbed scores_integrated_grad20_avgposref 20.013184 num negative: 37
avg_diff_zerod scores_integrated_grad20_avgposref 8.86813 num negative: 343

avg_diff_mean

In [17]:
def vis_examples_favoring_method_1(method_1_name, method_2_name,
                                   perf_lookup, imp_scores_fh,
                                   onehot_seq,
                                   num_to_plot=3):
    method_1_perfs = perf_lookup[method_1_name]
    method_2_perfs = perf_lookup[method_2_name]
    method_1_minus_method_2_perfs = method_1_perfs - method_2_perfs
    top_method_1_examples = [x[0] for x in sorted(enumerate(method_1_minus_method_2_perfs),
                                   key=lambda x: -x[1]) if x[1] > 0]
    
    method_1_scores = np.array(imp_scores_fh[method_1_name])
    method_2_scores = np.array(imp_scores_fh[method_2_name])
    
    print(len(top_method_1_examples))
    for ex_id in top_method_1_examples[::int(len(top_method_1_examples)/num_to_plot)]:
        print("Example_idx",ex_id,"pred",orig_preds[ex_id],
              method_1_name+"_diff",method_1_perfs[ex_id],
              method_2_name+"_diff",method_2_perfs[ex_id])
        print(method_1_name+" imp scores")
        viz_sequence.plot_weights((onehot_seq[ex_id]*method_1_scores[ex_id][:,None])[350:650],
                                  subticks_frequency=20)
        print(method_2_name+" imp scores")
        viz_sequence.plot_weights((onehot_seq[ex_id]*method_2_scores[ex_id][:,None])[350:650],
                                  subticks_frequency=20)

In [ ]:
vis_examples_favoring_method_1(method_1_name="scores_deeplift_genomicsdefault_dinucshuff",
                               method_2_name="scores_deeplift_rescale_dinucshuff",
                               perf_lookup=method_to_subbed_diffs,
                               imp_scores_fh=imp_scores_fh,
                               onehot_seq=onehot_seq)

In [ ]:
#visualize some examples where one method picks up stuff the other doesn't

vis_examples_favoring_method_1(method_1_name="scores_deeplift_rescale_dinucshuff",
                               method_2_name="scores_ism",
                               perf_lookup=method_to_subbed_diffs,
                               imp_scores_fh=imp_scores_fh,
                               onehot_seq=onehot_seq)